# Generating the Domain data corpus for domain adaptive fine-tuning

Initialization - Set Runtime type as GPU

This Notebook is part of the final Thesis.

The following objectives are achieved in this Notebook

1. To do Domain adaptive fine-tuning of the pre-trained Language models, a data corpus needs to be created. 
2. Downloadable contents from NCFE.ORG are used to create the data corpus for domain adaptation of the language models. 
3. The downloadable data are largely in the form of PDFs, and Presentations. The PDFs are all merged, converted to text, pre-processed, chunked, and then uploaded as Hugging Face dataset for Language Modeling (LM). 

In [1]:
# Make sure you have a GPU running
!nvidia-smi

Fri Dec 10 07:15:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Let's start by installing Haystack

# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1 -q 
!pip install git+https://github.com/deepset-ai/haystack.git -q
!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
!tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.

     |████████████████████████████████| 2.5 MB 8.9 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-9r1uyme_
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-9r1uyme_
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 14.1 MB 9.4 MB/s 
     |████████████████████████████████| 2.5 MB 13.1 MB/s 
     |████████████████████████████████| 51 kB 562 kB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 321 kB 39.1 MB/s 
     |████████████████████████████████| 359 kB 40.4 MB/s 
     |████████████████████████████████| 85 kB 4.3 MB/s 
     |████████████████████████████████| 981 kB 37.8 MB/s 
     |████████████████████████████████| 3.0 MB 44.3 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 5.6 MB 34.5 MB/s 
     |█████████

--2021-12-10 07:17:37--  https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
Resolving dl.xpdfreader.com (dl.xpdfreader.com)... 45.79.72.155
Connecting to dl.xpdfreader.com (dl.xpdfreader.com)|45.79.72.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23024720 (22M) [application/x-gzip]
Saving to: ‘xpdf-tools-linux-4.03.tar.gz’

xpdf-tools-linux-4. 100%[===================>]  21.96M  79.0MB/s    in 0.3s    

2021-12-10 07:17:38 (79.0 MB/s) - ‘xpdf-tools-linux-4.03.tar.gz’ saved [23024720/23024720]

xpdf-tools-linux-4.03/
xpdf-tools-linux-4.03/ANNOUNCE
xpdf-tools-linux-4.03/bin32/
xpdf-tools-linux-4.03/bin32/pdftotext
xpdf-tools-linux-4.03/bin32/pdfinfo
xpdf-tools-linux-4.03/bin32/pdftopng
xpdf-tools-linux-4.03/bin32/pdfimages
xpdf-tools-linux-4.03/bin32/pdftoppm
xpdf-tools-linux-4.03/bin32/pdftops
xpdf-tools-linux-4.03/bin32/pdfdetach
xpdf-tools-linux-4.03/bin32/pdffonts
xpdf-tools-linux-4.03/bin32/pdftohtml
xpdf-tools-linux-4.03/CHANGES
xpdf-tools-linux-

In [1]:
from haystack.modeling.utils import initialize_device_settings

devices, n_gpu = initialize_device_settings(use_cuda=True)

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [2]:
!pip install datasets -q

     |████████████████████████████████| 298 kB 8.2 MB/s 
     |████████████████████████████████| 61 kB 300 kB/s 
     |████████████████████████████████| 243 kB 40.0 MB/s 
     |████████████████████████████████| 132 kB 42.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.7.0 requires huggingface-hub==0.0.8, but you have huggingface-hub 0.2.1 which is incompatible.


In [1]:
import datasets
from datasets import load_dataset

In [2]:
# required for pdf merge
! pip install pypdf2 -q

     |████████████████████████████████| 77 kB 3.9 MB/s 


In [ ]:
# # required for pdf converter
# !apt-get install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils \
#      pstotext tesseract-ocr \
#      flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig libasound2-dev libpulse-dev

In [3]:
!pip install git+https://github.com/deanmalmgren/textract -q

     |████████████████████████████████| 106 kB 12.5 MB/s 
     |████████████████████████████████| 111 kB 41.8 MB/s 
     |████████████████████████████████| 69 kB 2.9 MB/s 
     |████████████████████████████████| 5.6 MB 19.8 MB/s 
     |████████████████████████████████| 10.1 MB 36.7 MB/s 
     |████████████████████████████████| 32.8 MB 1.4 MB/s 
     |████████████████████████████████| 103 kB 51.9 MB/s 
     |████████████████████████████████| 2.0 MB 37.8 MB/s 
     |████████████████████████████████| 128 kB 28.4 MB/s 
     |████████████████████████████████| 112 kB 37.7 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
     |████████████████████████████████| 149 kB 46.6 MB/s 
     |████████████████████████████████| 70 kB 7.5 MB/s 
     |████████████████████████████████| 339 kB 47.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tox 3.24.4 require

In [1]:
# Here are the imports we need
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
import numpy as np
import pandas as pd
import regex as re
import os
import torch
import json
import sys
import time
import datetime
import random
import collections
from pathlib import Path

In [3]:
# Utility Function - Show random elements
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

# Merge Multiple PDFs

First of all, custom data needs to be created for the Language model pre-training. This is done by consolidating all the downloadable contents from NCFE.org and then combining them all into a text document using available tools. 

In [52]:
# Merge all pdf files in a current directory 

from glob import glob
from PyPDF2 import PdfFileMerger

def pdf_merge():
    ''' 
    Merges all the pdf files in current directory 
    Creates a new merged pdf file in the current directory
    '''
    merger = PdfFileMerger(strict=False)
    allpdfs = [a for a in glob("*.pdf")]
    for pdf in allpdfs:
        print (pdf)
        merger.append(pdf)
    #[merger.append(pdf) for pdf in allpdfs]
    with open("Merged_pdfs.pdf", "wb") as new_file:
        merger.write(new_file)

In [5]:
# Change Directory to merge pdfs from 

os.chdir('/content/drive/MyDrive/FinLitQA/NCFE_Content')
print(os.getcwd() + "\n")

/content/drive/MyDrive/FinLitQA/NCFE_Content



In [35]:
# Merges all the pdfs in the current directory and create a new merged file in the current directory
# Correctly specify the current directory before merge
# Change Directory to merge pdfs from 

path_list = ['/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Booklets',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Derivatives',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Equity',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_FAQ',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Handbooks',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Insurance',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Mutual_Fund',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Others',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Pensions',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Target_Group_Booklets',
             '/content/drive/MyDrive/FinLitQA/NCFE_Content/Other_Books'
             ]

path_list1 = ['/content/drive/MyDrive/FinLitQA/NCFE_Content/Other_Books']


In [56]:
for p in path_list1:
    print (f'Now merging documents in {p}.')
    os.chdir(p)
    pdf_merge()

Now merging documents in /content/drive/MyDrive/FinLitQA/NCFE_Content/Other_Books.
Law.pdf
Financial_planning.pdf
Banking.pdf
Banking_Study.pdf
Fundamentals.pdf


In [57]:
# After creating merged Pdf documents for individual sections copy them all to one folder to create the master merged file

import shutil
import datetime

now = str(datetime.datetime.now())[:19]
now = now.replace(":","_")

# Current Directory 
%cd /content/drive/MyDrive/FinLitQA/NCFE_Content

# Create Directory to save the model
!mkdir Merged_pdfs

ctr = 1
for p in path_list:
    src_dir= p+"/"+"Merged_pdfs.pdf"
    print (src_dir)
    dst_dir="/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_"+str(now)+str(ctr)+".pdf"
    print (dst_dir)
    shutil.copy(src_dir,dst_dir)
    ctr+=1

/content/drive/MyDrive/FinLitQA/NCFE_Content
mkdir: cannot create directory ‘Merged_pdfs’: File exists
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Booklets/Merged_pdfs.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_2021-12-10 08_58_281.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Derivatives/Merged_pdfs.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_2021-12-10 08_58_282.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Equity/Merged_pdfs.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_2021-12-10 08_58_283.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_FAQ/Merged_pdfs.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_2021-12-10 08_58_284.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Handbooks/Merged_pdfs.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_2021-12-10 08_58_285.pdf
/content/drive/MyDrive/FinLitQA/NCFE_Content/NCFE_Insurance/Merged_pdfs.pdf
/con

In [59]:
# Merge all the documents in the Merged Pdfs folder to create a master pdf file

os.chdir("/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs")
pdf_merge()

Merged_2021-12-10 08_58_281.pdf
Merged_2021-12-10 08_58_282.pdf
Merged_2021-12-10 08_58_283.pdf
Merged_2021-12-10 08_58_284.pdf
Merged_2021-12-10 08_58_285.pdf
Merged_2021-12-10 08_58_286.pdf
Merged_2021-12-10 08_58_287.pdf
Merged_2021-12-10 08_58_288.pdf
Merged_2021-12-10 08_58_289.pdf
Merged_2021-12-10 08_58_2810.pdf
Merged_2021-12-10 08_58_2811.pdf


# Convert Merged pdf to text

In [60]:
# Using File Converter
from haystack.nodes import PDFToTextConverter

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["de","en"])

final_merged_file_path = '/content/drive/MyDrive/FinLitQA/NCFE_Content/Merged_PDFs/Merged_pdfs.pdf'

merged_text = converter.convert(file_path=final_merged_file_path, meta=None)

In [82]:
# merged_text, len(merged_text)

In [61]:
type(merged_text)

list

In [62]:
merged_text_clean = merged_text.copy()

In [ ]:
# merged_df = pd.DataFrame([merged_text])
# merged_df

# Cleaning and Pre-processing

In [63]:
import regex as re

def clean_text(text):
    """Returns a lower-cased string with punctuations and special characters removed.
    Returns:
        processed_text: str
    ----------
    Arguments:
        text: str of text
    """
    text = str(text)
    #Substitute punctuations and special characters for a space
    x = re.sub('[…“”%!&"@#()\-\*\+,/:;<=>?@[\]\^_`{\}~]', ' ', text)
    #Remove periods
    y = re.sub('[\.\']', "", x)
    #Remove Newlines
    y = re.sub("\n", "", text)
    text = text.strip().replace('\n', ' ')
    # Apply lower-case
    cleaned_text = text.lower()

    return cleaned_text

In [ ]:
# merged_df['content_clean'] = merged_df['content'].apply(clean_text)
# merged_df.head()

**Cleaning**

- `clean_empty_lines` will normalize 3 or more consecutive empty lines to be just a two empty lines
- `clean_whitespace` will remove any whitespace at the beginning or end of each line in the text
- `clean_header_footer` will remove any long header or footer texts that are repeated on each page

In [64]:
# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

def pre_process(doc_txt):
    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=True,
        split_by="word",
        split_length=200,
        split_respect_sentence_boundary=True
    )
    processed_text = preprocessor.process(doc_txt)
    return processed_text


In [65]:
merged_text_clean

Output hidden; open in https://colab.research.google.com to view.

In [66]:
merged_dict = merged_text_clean[0]

In [30]:
merged_dict

Output hidden; open in https://colab.research.google.com to view.

In [67]:
merged_dict.update({'content': clean_text(merged_dict['content'])}) 

In [68]:
merged_dict

Output hidden; open in https://colab.research.google.com to view.

In [69]:
# Pre-process the merged-text using the PreProcessor utility

ncfe_docs = pre_process(merged_dict)

print(f"n_docs_input: 1\nn_docs_output: {len(ncfe_docs)}")

WARNING - haystack.nodes.preprocessor.preprocessor -  One or more sentence found with word count higher than the split length.


n_docs_input: 1
n_docs_output: 4003


In [70]:
len(ncfe_docs), ncfe_docs[:5], ncfe_docs[-5:]

(4003,
 [{'content': 'fundamental messages in financial literacy savings national centre for financial education\nsavings   one by one makes a bundle you don\'t need much money to start saving. whether you are a student, a farmer, a teacher, nurse, banker, market vendor, taxi driver or a business person, you can always put a little money aside. when you save regularly, your money will "grow". spend less to save more you can save by spending less. cutting down on consumption, such as alcohol and on buying new clothes for every function, enables you to save more money to provide for you and your family\'s future. encourage your children to save parents should teach children to start saving for a purpose while they are still young. this helps children to understand the value of money and to develop a savings culture at an early age. you can help a child save money in a small tin or box (piggy bank). save to avoid unnecessary debt savings are the best way to pay for day-today costs like sc

In [71]:
import pandas as pd
ncfe_data_df = pd.DataFrame.from_dict(ncfe_docs)

In [72]:
ncfe_data_df = ncfe_data_df["content"]

In [73]:
ncfe_data_df.head()

0    fundamental messages in financial literacy savings national centre for finan...
1    for example, if you start saving for your children's\nsavings education earl...
2    it is your right to enjoy your money if you plan and save towards such luxur...
3    if you are an employee, you can also ask your employer to transfer part of y...
4    in exchange, the insurance company agrees to pay you a certain amount of mon...
Name: content, dtype: object

# Write Data to Drive

In [74]:
# Copy CSV file to Google Drive

ncfe_data_df.to_csv('ncfe_data.txt', header=True, index=True, sep='\t', mode='a')

In [75]:
# Write to drive location

!cp ncfe_data.txt "/content/drive/MyDrive/FinLitQA/NCFE_Content"

# Prepare Data for Training

In [76]:
from sklearn.model_selection import train_test_split
ncfe_data_train, ncfe_data_valid = train_test_split(ncfe_data_df, test_size=0.2, random_state=100, shuffle=True)

In [77]:
# Write data to file
# Created three new text files in the FinLitQA/NCFE_Content folder 
# ncfe_data.txt , ncfe_data_train.txt, ncfe_data_valid.txt

import csv

ncfe_data_path = '/content/drive/MyDrive/FinLitQA/NCFE_Content/ncfe_data.txt'
ncfe_data_train_path = '/content/drive/MyDrive/FinLitQA/NCFE_Content/ncfe_data_train.txt'
ncfe_data_valid_path = '/content/drive/MyDrive/FinLitQA/NCFE_Content/ncfe_data_valid.txt'

ncfe_data_df.to_csv(ncfe_data_path ,index=False,header=False, sep="\t")
ncfe_data_train.to_csv(ncfe_data_train_path, index=False,header=False, sep="\t")
ncfe_data_valid.to_csv(ncfe_data_valid_path, index=False,header=False, sep="\t")

In [78]:
print (ncfe_data_df.shape, ncfe_data_train.shape, ncfe_data_valid.shape)

(4003,) (3202,) (801,)


In [79]:
from datasets import load_dataset
finlit_data = load_dataset("text", data_files={"train": ncfe_data_train_path,
                                             "validation": ncfe_data_valid_path})

finlit_data, finlit_data.column_names

WARNING - datasets.builder -  Using custom data configuration default-3f4ed03ae061731f


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-3f4ed03ae061731f/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4690
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1220
    })
}),
 {'train': ['text'], 'validation': ['text']})

In [80]:
show_random_elements(finlit_data["train"])

,text
0,"""market capitalization companies with market cap in excess of us$ 3.5 billion public float at least 50% public float financial viability companies should have four consecutive quarters of positive earnings"
1,"""applications for such permission should be addressed to the chief, publishing and multimedia service, information division, fao, viale delle terme di caracalla, 00100 rome, italy or by e-mail to copyright@ fao.org i"
2,"""every bank maintains a credit investigation department at its head office and main offices in larger cities to collect information regarding the financial position of its borrowers. at other centres, credit investigation is performed by branch managers. the credit information is collected through the following sources: 1. credit information bureau: the reserve bank of india has established within itself a credit information bureau which collects credit information from the banks under section 45-c (1) of the reserve bank of india act, 1934. banks are required to furnish such credit information in respect of credit limits of ` 5 lakhs and over in case of secured advances and ` 1 lakh and over in case of unsecured advances. they mention the nature of facility, security and charge along with outstanding balance. after consolidating such information in respect of each customer, the reserve bank supplies to the applicant-banks information relating to the total limits sanctioned to and the number of banks dealing with a party. thus, the banks can"
3,"procedure of opening and operating of deposit account iii) in case of payment by mistake, bank can collect such amount from the account holder. 3.1.6. specimen signature : a new customer is required to sign an account opening form and a specimen signature slip/card in the presence of an authorised officer who attests his specimen signature at the time of opening an account. the specimen signature cards are preserved by the bank. the bank compares the signature on the withdrawal slip with the specimen signature. importance : i) the signature on the card can be verified at the time of withdrawl. ii) it provides payment of money to right person. iii) one cannot say that he has not withdrawn money from the bank. 3.1.7. nomination : in case of death of an account holder, the problem of whom to pay the balance arises. now-a-days banks take nomination in writing in case of all accounts. in case of the death of an account holder the bank pays the money to the nominee and closes the account. a nomination made by a depositor in the prescribed manner will confer the nominee right to receive the amount of deposit from the bank."
4,"10,000 more than rs. 10,000 less than rs. 10,000 more than rs. 10,000 commission payable nil rs. 150 nil rs."


In [81]:
show_random_elements(finlit_data["validation"])

,text
0,"""as time passes, the proportion of payment allocated to the principal increases and interest payment decreases. notice that in the 20th year, the total interest payment is ` 26,096 and the principal amount paid is ` 4,99,905. allocation of interest versus principal per year on ` 47,02,500 at annual interest rate of 9.5% for a 10 year loan"
1,"syllabus module iii, elective paper 9.1: banking law and practice (100 marks) level of knowledge: expert knowledge objective: to acquire specialized knowledge of law and practice relating to banking detailed contents: 1. overview of banking system 2. regulatory framework and compliances a. provisions of rbi act 1935, banking regulation act 1949, prevention of money laundering act, 2002. b. government and rbi's powers opening of new banks and branch licensing constitution of board of directors and their rights banks share holders and their rights crr and slr concepts cashcurrency management winding up - amalgamation and mergers powers to control advances - selective credit control monetary and credit policy audit and inspection supervision and control - board for financial supervision its scope and role disclosure of accounts and balance sheets submission of returns to rbi, corporate governance."""
2,"""b omits to see this"
3,"debentures these are similar to bonds, but are issued by companies. there are different kinds of debentures, which can be offered. they are as follows: non convertible debentures (ncd) total amount redeemed by the issuer partially convertible debentures (pcd) part is redeemed and part is converted to equity shares with or without the option to the investor fully convertible debentures (fcd) whole value is converted into equity. the conversion price is stated when the instrument is issued company fixed deposits these are similar to bank fixed deposits, issued by companies to attract small investors. however, they are repayable only on maturity. they offer attractive interest rates, however, they are insecured. ratings are guide to investor for safety. mutual funds a mutual fund is a body corporate that pools money from many investors and invests the money in stocks, bonds, short-term money-market instruments, other securities or assets, or some combination of these investments."""
4,"wave characteristic final leg in the direction of the dominant trend news is almost universally positive and everyone is bullish many average investors finally buy in, right before the top volume is often lower in wave five than in wave three momentum indicators start to show divergences at the end of a major bull market, bears may very well be ridiculed wave a corrections are typically harder to identify than impulse moves fundamental news is usually still positive analysts see the drop as a correction in a still-active bull market some technical indicators that accompany wave a include o increased volume o rising implied volatility in the options markets and o possibly a turn higher in open interest in related futures markets"
